# Tutorial 4 - Running a simulation with the MLFF 

You made it this far. Good job! Now the final test, can we run a simulation with the MLFF? For MLFFs the long term stability is determining whether or not it is a good force field, so we will simulate for some time. But first, lets set up our environment. 

## Setting up the environment 



In [ ]:
import torch
from torch_geometric.data import Data

def create_datapoint_from_coll(coll):
    """
    Create a single PyG Data object from a molecule collection-like object.
    Assumes:
        - coll.get_atom_coords_list() returns shape (N, 3)
        - coll.get_atom_numbers() returns shape (N,)
    """
    pos = torch.tensor(coll.coordinates, dtype=torch.float64)  # shape (N, 3)
    z = torch.tensor(coll.get_atom_numbers(), dtype=torch.float64)        # shape (N,)

    num_atoms = pos.shape[0]
    row, col = torch.meshgrid(
        torch.arange(num_atoms), torch.arange(num_atoms), indexing="ij"
    )
    edge_index = torch.stack([row.flatten(), col.flatten()], dim=0)       # shape (2, N^2)

    edge_vec = pos[edge_index[0]] - pos[edge_index[1]]                    # (num_edges, 3)
    edge_attr = edge_vec.norm(dim=1, keepdim=True)                        # (num_edges, 1)
    edge_attr = torch.cat([edge_vec, edge_attr], dim=1)                   # (num_edges, 4)

    return Data(
        pos=pos,
        z=z.unsqueeze(1).double(),      # [N, 1]
        edge_index=edge_index,          # [2, num_edges]
        edge_attr=edge_attr,            # [num_edges, 4]
        edge_vec=edge_vec               # [num_edges, 3]
    )

In [ ]:
from mond.molecule import MoleculeCollection
from mond.utils import create_molecule_from_smiles, random_pose

bounding_box = [10, 10, 10] # In angstrom

mol_smiles = "CO"
molec = create_molecule_from_smiles(mol_smiles)
new_coords = random_pose(molec.coordinates, bounding_box)
molec.set_mol_conf_coordinates(new_coords)
coll = MoleculeCollection([molec])

In [ ]:
data_point = create_datapoint_from_coll(coll)

In [ ]:
data_point

## Verlet Integration 

We are going to solve the equations of motion (EOMs) with the Verlet integration. For the sake of completeness you could add a Thermostat, e.g. the Berendsen-Thermostat. AI Coding tools do a good job here. 

In [ ]:
def get_forces_mlff(data_point, model):
    forces = model(data_point)
    return forces.detach().numpy()

In [ ]:
def velocity_verlet_step_mlff(
    model,
    data_point, 
    positions, 
    velocities, 
    forces, 
    masses, 
    box_lengths, 
    dt, 
    periodic_boundary):
    """
    Ein Velocity-Verlet-Schritt mit gleicher Masse für alle Teilchen.
    
    Parameters
    ----------
    positions : (N, 3)
    velocities : (N, 3)
    forces : (N, 3)
    masses : (N,) - Massen der Teilchen
    epsilon : float
    sigma : float
    """
    masses = np.array(masses)
    acc = forces / masses[:, np.newaxis]


    positions += velocities * dt + 0.5 * acc * dt**2
    if periodic_boundary == True: 
        positions %= box_lengths  # PBC

    new_forces = get_forces_mlff(
        data_point=data_point, 
        model=model)
    new_acc = new_forces / masses[:, np.newaxis]
    velocities = velocities + 0.5 * (acc + new_acc) * dt
    assert np.sum(np.isnan(velocities))==0, "Velocities contain NaN values"
    return positions, velocities, new_forces

In [ ]:
import numpy as np 

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary

from tqdm import tqdm

from mond.molecule import Molecule, MoleculeCollection
from mond.utils import create_molecule_from_smiles, random_pose
from mond.simulation.aimd import init_traj, append_to_traj, initialize_velocities
from mond.utils import get_atomic_masses

bounding_box = [10, 10, 10] # In angstrom

mol_smiles = "CO"
molec = create_molecule_from_smiles(mol_smiles)
new_coords = random_pose(molec.coordinates, bounding_box)
molec.set_mol_conf_coordinates(new_coords)
coll = MoleculeCollection([molec])

traj_file="methanol_mlff_data_aug_traj.xyz"
box_lengths = np.array(bounding_box)
atom_symbols = coll.get_atom_symbols()
masses = get_atomic_masses(atom_symbols)
dt = 0.05 #fs
temp_init = 298.5 #temperature intialization
temp_thermostat =0
periodic_boundary = False
num_steps = 10000 
positions = coll.get_atom_coords_list()
print("Initial positions")
print(positions)
#model 
model_path = "mpnn_full_model_data_aug_SiLU_5000.pth"
model = torch.load(model_path, map_location="cpu", weights_only=False)
#intialization
data_point = create_datapoint_from_coll(coll)
velocities = initialize_velocities(atom_masses=masses, temp=temp_init, remove_drift=True)
new_forces =  get_forces_mlff(
    data_point=data_point,
    model = model)
# initialize trajectory 
print(new_forces)
init_traj(traj_file)
#one step
for i in tqdm(range(num_steps), desc="Running AIMD"): 
    
    positions, velocities, new_forces = velocity_verlet_step_mlff(
        model=model,
        data_point=data_point, 
        positions=positions, 
        velocities=velocities, 
        forces=new_forces, 
        masses=masses, 
        box_lengths=box_lengths, 
        dt=dt, 
        periodic_boundary=periodic_boundary
    )
    append_to_traj(
        traj_file = traj_file, 
        positions = positions.tolist(), 
        symbols=atom_symbols
    )
    coll.coordinates = positions
    data_point = create_datapoint_from_coll(coll)